# Mapper algorithm
The mapper algorithm is a "modular" algorithm for creating a low-dimensional representation (usually a graph) of a large and/or high-dimensional (point cloud) dataset while preserving interesting topological characteristics. The results of the algorithm depend on three choices that we can make for it:

1. A (combination of) *filter* function(s), mapping the dataset to a metric space (called the *parameter space*) based on some distance/similarity metric (e.g. height of points or angle relative to some center).
2. An *open cover* of the parameter space, usually consisting of overlapping open intervals. The pre-images of the cover sets under the filter determine an open cover of the original point cloud.
3. A *clustering algorithm* to divide the covering sets into clusters/"connected components". The graph will be constructed based on overlap of these clusters.

The default choice for the parameter space is $\mathbb{R}$, which produces a graph as a result. However, Mapper can also be extended to $S^1$ (producing a graph with cycles) or $\mathbb{R}^M$ (producing a simplicial complex of dimension $\leq M$). For specific choices of filter and parameter space, cover, and clustering algorithm, we retrieve other well-known TDA algorithms for representing high-dimensional data, such as density clustering trees, disconnectivity graphs, and Reeb graphs; mapper is essentially a generalization of these techniques.

### Filters, covers & clusters
The *filter* function essentially reduces data points to only their relevant characteristics (e.g. reducing a 3d point $(x,y,z)$ to only its height $y$). Given a set $X$ of $N$ points, the filter is a function $f: X \to Z$ (usually the parameter space is $Z = \mathbb{R}$, but it can also be $\mathbb{R}^2$ or $S^1$) which assigns a value to each of the $N$ points. 

We then partition the *parameter space* $Z$ based on the range $I$ of $f$; this is typically done using a set $S$ of smaller, overlapping intervals, defined by a length $l$ and a percentage of overlap $p$. For example, if $I = [0,2]$, $l = 1$ and $p = 2/3$, then $S = \{[0,1], [1/3, 4/3], [2/3, 5/3], [1, 2]\}$.

Then, we use this partitioning of $Z$ to derive a cover $\mathcal{U}$ of $X$, defined by $\mathcal{U} = \bigcup_{I_j \in S} f^{-1}(I_j)$. We also divide the *covering sets* $X_j = f^{-1}(I_j)$ further into clusters $X_{jk}$, which can be thought of as representing connected components. Ultimately, each cluster will be a vertex in our complex and represents the points within it. We draw an edge between clusters $X_{jk}$ and $X_{lm}$ if their intersection is non-empty, i.e. if there is a point which exists in both clusters (see also Figure 1 in the mapper paper). Note that a clustering algorithm usually assigns each point within an $X_j$ to *one* unique cluster; this means there will be no edges between clusters within the same covering set $X_j$.

But how do we divide these $X_j$ into clusters? That's using a user-defined *clustering algorithm*. Mapper does not place any requirements on this, so any (domain-specific) algorithm will work. This appears to be the part of the algorithm that Mapper is most sensitive to, so it's a good idea to test a couple different clustering algorithms, and in particular we should try a variable amount of clusters for different covering sets.

### Example
An example of how the filter and clustering could work for our data, assuming that the point clouds are rotated such that the tree always grows in the $+y$ direction:
- For the filter $f$, we use $f(x,y,z) = y$, i.e. each point is mapped to its height above the ground.
- We then partition the resulting range (which is $[min \ height, \  max \ height]$) into intervals with overlap; the values of $l$ and $p$ will have to be tuned.
- We then cluser points into balls, (rotated) boxes, or similar shapes. The key requirement is that *clusters that belong to the same leader/support branch should overlap, while clusters that belong to different branches should not*. This requires tuning the size of the balls/boxes.
- From these clusters and their overlap, we can construct the mapper graph.
- In the graph, we should be able to identify the four key parts of the tree:
    - There should be only two intervals ($\textcolor{red}{red}$ and $\textcolor{orange}{orange}$ in the example) which have only one connected component. The one with many clusters ($\textcolor{orange}{orange}$) is the support, while the one with only a few clusters ($\textcolor{red}{red}$) is the trunk.
    - The leaders are formed by long, "straight" connected components spanning multiple intervals. Side branches are clusters which have a sharp angle to one of these leader "chains".
    - Of course, one of the biggest challenges will be finding better heuristics for identifying parts of the tree which are robust against various types of noise.

This would produce a result that looks something like this:

![mapper graph](Mapper_Mockup_Real.png)

Again, note that there are no connections between points within the same covering set (color), so the result doesn't quite look like a tree; our challenge is to find a good filter, cover and clustering algorithm that *does* produce a tree-like shape, and ideally one where we can actually label the branches too.

# Implementation
Assumption: we are given a point cloud dataset $X$ of $N$ points, each of which are arrays of the form [x,y,z].

Things we can tweak in the algorithm:
1. The filter function
2. The partition function
3. The clustering algorithm

For now we'll only consider $Z = \mathbb{R}$ as a parameter space; the algorithm can be extended to higher dimensions to produce not just graphs (1-simplicial complexes) but higher dimensional simplicial complexes as well.

In [12]:
### IMPORTS & GENERAL SETTINGS ###
import numpy as np
from tqdm import tqdm
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KernelDensity, NearestNeighbors
from scipy.spatial.distance import cdist

### Filter
**Coordinate**
Function is just filtering either a $x$, $y$ or $z$-coordinate from all points in the point_cloud. 

**Estimate_Local_Point_Density**
This function estimates the density around each point in a point cloud using either the kernels 'gaussian', 'tophat', 'epanechnikov', 'exponential', 'linear' or 'cosine':
$f_e(x) = C_\epsilon \sum_{y \in P} exp(- \frac{d(x,y)^2}{\epsilon})$.
The width of the kernel (bandwidth) $\epsilon$ controls the smoothness of the density function; larger 'width' leads to more global estimates taking into account point with a greater to the concerning point which leads to smoother function. The opposite is true for smaller widht $\epsilon$. High $f_{\epsilon}(x)$ point lies in thus in dense space and low $f_{\epsilon}(x)$ lie in a sparse space or near the boundary.

**Eccentrity_Filtering**
The eccentrity filtering algorithm measures the generalized distance from point $x$ to all other points: $E_p(x) = (\frac{1}{|X|} \sum_{y \in X} ||x-y||^p)^{1/p}$. It gives a global measure of centrality and captures geometric or topoloical "depth" of point in the cloud. The input parameter $p$ controls the level of sensitivity for outliers; high $p$ is highly sensitive while low $p$ is less sensitive for outliers. Low $E_p(x)$ means the concerning point is a central point while high $E_p(x)$.

**KNN_Distance**
The kNN filtering algorithm measures the Euclidean distance to the $k^{th}$ nearest neighbor of point $x$: $f_k(x) = ||x - kNN_k(x)||$. It gives an interpretation if the point is surrounded closely by other points (high $f_k(x)$) or if the points lays in sparse space (low $f_k(x)$). 

In [13]:
### FILTER FUNCTION ###
# Input: depends on exact function; usually either the coordinates of one point 
#        or the distances between one point and all other points (for e.g. density or eccentricity filtering)
# Output: a real value

'''
Timo's proposal for preprocessing and filtering (step-by-step plan):
 -- Statistical- or Radius Outlier Removal to remove <= 0.05% of the points to be sure the point cloud still maintains the side branches, 
 but the noise in the point cloud is removed.
 -- Superpoint Selection to downsample the number of points. (~100 - 1000 points)
 -- One of the four filtering algorithms to lower dimension of each point from R^3 to R. 
'''

def filter_coordinate(point, coordinate):
    """Filters one coordinate of a point either (x,y or z value)
    x-value is coordinate 0
    y-value is coordinate 1
    z-value is coordinate 2 """
    return point[coordinate]

def filter_height(point):
    """Shortcut for the y-coordinate filter."""
    return filter_coordinate(point, 1)

def estimate_local_point_density(point_cloud, kernel_type, bandwidth):
    """
    Kernel density estimator to assign higher values to points in dense regions and
    lower values to sparse or boundary regions.

    Parameters:
    point_cloud: array of shape (N, 3)
    kernel_type: the used kernel ('gaussian', 'tophat', 'epanechnikov', 'exponential', 'linear', 'cosine')

    Returns:
    Smoothed point cloud (array of shape (N,))
    """
    kde = KernelDensity(kernel=kernel_type, bandwidth=bandwidth).fit(point_cloud)
    smoothed_point_cloud = kde.predict(point_cloud)
    return smoothed_point_cloud

def eccentrity_filtering(point_cloud, p=2):
    """
    Compute eccentricity for each point in a 3D point cloud X.

    Parameters:
    point_cloud: array of shape (N, 3)
    p: moment order (int or inf)

    Returns:
    Eccentricity for each point (array of shape (N,))
    """
    # Compute pairwise distance
    D = cdist(point_cloud, point_cloud)

    if p == np.inf:
        ecc = np.max(D, axis=1)
    else:
        ecc = (np.sum(D**p, axis=1) / point_cloud.shape[0])**(1/p)

    return ecc

def knn_distance(point_cloud, k=10):
    """
    Compute distance th the k-th nearest neighbor for each point.
    f_k: R^3 -> R

    Parameters:
    point_cloud: array of shape (N, 3)
    k: number of nearest neighbors (int)

    Returns:
    Distance to k-th nearest neighbor, (array of shape (N,))
    """
    nbrs = NearestNeighbors(n_neighbors=k+1).fit(point_cloud)
    distances, _ = nbrs.kneighbors(point_cloud)

    return distances[:, k]

def branch_direction_filter(branch_direction):
    '''
    Filter based on the branch growth direction vector of the point; the return value is the y-component of the direction vector.
    Filter values will be high for points where the branch grows near-vertical, and low for points where the growth is near-sideways. 
    '''
    return abs(branch_direction[1])


### Cover
For now we use the default overlapping intervals.

In [14]:
### COVERING FUNCTION ###
# Covers the image of the filter function with overlapping intervals
# Input: minimum filter function value, max filter function value, interval length l, interval overlap percentage p
# Output: a dictionary whose keys are the overlapping intervals and whose values are empty arrays.
# Example with min = 0, max = 2, l = 1 and p = 0.5: {(0,1): [], (0.5,1.5): [], (1, 2): []}
def cover_intervals(fmin, fmax, l, p):
    '''If the filter function f has values between fmin and fmax, this function covers the range [fmin, fmax] 
    with intervals of length l which overlap for a percentage 0 < p < 1.
    Example output for fmin = 0, fmax = 2, l = 1 and p = 0.5: {(0,1): [], (0.5,1.5): [], (1, 2): []}'''
    output = {}
    epsilon = 1e-8
    overlap_length = p*l
    I_start = fmin - epsilon  # Starting point of current interval. 
                              # We subtract a small constant so fmin itself is also included in the first interval.
    output[(I_start, I_start + l)] = []

    # Add new intervals until we've covered the whole range [fmin, fmax]
    # Note: the last interval may extend past fmax, but this is fine.
    while I_start + l < fmax:
        I_start += l - overlap_length
        output[(I_start, I_start + l)] = [] # Add interval to output dict with empty array as value

    return output

# TEST
cover = cover_intervals(0, 5, 1.2, 0.3)
print(cover)

{(-1e-08, 1.19999999): [], (0.8399999899999999, 2.03999999): [], (1.6799999899999998, 2.87999999): [], (2.5199999899999996, 3.71999999): [], (3.3599999899999995, 4.55999999): [], (4.199999989999999, 5.3999999899999995): []}


In [15]:
### COVER POINT CLOUD ACCORDING TO FITLER FUNCTION & COVER IN PARAMETER SPACE ###
# Cover the original point cloud data according to the filtering function and the above cover of the parameter space.
# Input: point cloud dataset X
# Output: dictionary whose keys are the intervals in Z as found above, 
# and whose values are the points that are mapped to these intervals by the filter function.
# Note: a point may occur in multiple intervals (in that case they overlap).
# Example output if X = {a, b, c, d} with filter values respectively {0.4, 0.7, 1.3, 1.8}:
# {(0,1): [a, b], (0.5, 1.5): [b, c], (1, 2): [c, d]}
from copy import deepcopy

def apply_covering(X, filter_values, cover_dict, color_values = []):
    '''Cover point cloud dataset X whose points have filter values filter_values, using the intervals in cover_dict.keys().
    Returns two dicts, both with the cover intervals as keys: one with the actual points, and one with the corresponding filter values.
    If color_values != [], the filter values in color_values (which may be based on a different filter) are used instead of filter_values in the second dict.
    Note: the indices in X and filter_values are expected to match.'''
    output_pts = deepcopy(cover_dict)
    output_vals = deepcopy(cover_dict)
    intervals = list(cover_dict.keys())
    use_color_vals = (color_values != [])

    for i, filter_value in tqdm(enumerate(filter_values)):
        # Add point to each interval that its filter value falls into
        for interval in intervals:
            if interval[0] < filter_value and filter_value < interval[1]:
                output_pts[interval].append(X[i])
                output_vals[interval].append(color_values[i] if use_color_vals else filter_value)
    
    return output_pts, output_vals

# TEST
cover_l = 1
cover_p = 0.6
X = [[0, 0.4, 0.2], [0.3, 0.7, 1.0], [1.6, 1.3, 0.2], [2.4, 1.8, -0.4]]
filter_values = [filter_height(point) for point in X]
color_values = [12, 37, 42, -5]

fmin = min(filter_values)
fmax = max(filter_values)

cover_dict = cover_intervals(fmin, fmax, cover_l, cover_p)
print("Cover of parameter space: ", cover_dict)
X_cover, X_cover_vals = apply_covering(X, filter_values, cover_dict, color_values)
print("Cover of X: ", X_cover)
print("Corresponding values: ", X_cover_vals)

Cover of parameter space:  {(0.39999999, 1.39999999): [], (0.7999999900000001, 1.79999999): [], (1.1999999900000002, 2.1999999900000002): []}


4it [00:00, ?it/s]

Cover of X:  {(0.39999999, 1.39999999): [[0, 0.4, 0.2], [0.3, 0.7, 1.0], [1.6, 1.3, 0.2]], (0.7999999900000001, 1.79999999): [[1.6, 1.3, 0.2]], (1.1999999900000002, 2.1999999900000002): [[1.6, 1.3, 0.2], [2.4, 1.8, -0.4]]}
Corresponding values:  {(0.39999999, 1.39999999): [12, 37, 42], (0.7999999900000001, 1.79999999): [42], (1.1999999900000002, 2.1999999900000002): [42, -5]}


### Clustering
For now we use k-means.
Major point of improvement: use different k for different covering sets based on the nr of points (and maybe also the shape?) of the covering set.

In [16]:
### CLUSTERING ALGORITHM ###
# Clusters the points in a covering set of X according to some clustering algorithm.
# Input: subset of points in X, additional parameters determining size and shape of clusters
# Output: an array/dictionary where each entry represents a cluster (and whose value is a list containing all points in that cluster).

from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering

def cluster_kmeans(covering_set, covering_set_vals, k=20):
    '''Clusters the points in the covering set using the k-means algorithm.
    Output: two dictionaries with the k cluster centroids as keys: one with the points belonging to each cluster as values, 
    and the other with the average filter value of the points in the cluster as value'''
    kmeans = KMeans(n_clusters=k, random_state=0).fit(covering_set)
    centroids = kmeans.cluster_centers_
    labels = kmeans.labels_  # For each point in covering_set, gives the index that that point belongs to

    output_pts = {}
    output_vals = {}
    for i in range(k):
        output_pts[tuple(centroids[i])] = covering_set[labels == i]
        output_vals[tuple(centroids[i])] = np.mean(covering_set_vals[labels == i])

    return output_pts, output_vals



def cluster_linkage(covering_set, covering_set_vals, distance_threshold=0.5):
    '''Clusters the points in covering_set using agglomerative clustering with a distance threshold instead of a fixed number of clusters.
    Returns two dictionaries with the cluster centroids as keys: one with the points belonging to each cluster as values, 
    and the other with the average filter value of the points in the cluster as value'''
    # Note: linkage could be made a parameter
    linkage = AgglomerativeClustering(n_clusters=None, linkage='ward', distance_threshold=distance_threshold).fit(covering_set)
    labels = linkage.labels_ 

    output_pts = {}
    output_vals = {}
    # For each cluster, calculate the cluster centroid as the mean of the points in that cluster
    # and assign all points in the cluster to that centroid in the output dictionary.
    for i in np.unique(labels):
        cluster_points = covering_set[labels == i]
        centroid = np.average(cluster_points, axis=0)
        output_pts[tuple(centroid)] = cluster_points
        output_vals[tuple(centroid)] = np.mean(covering_set_vals[labels == i])

    return output_pts, output_vals



def cluster_dbscan(covering_set, covering_set_vals, eps=0.5, min_samples=3):
    '''
    Clusters the points in the covering set using the DBSCAN algorithm.

    Parameters
    ------------
    eps : max distance between two samples for one to be considered in the neighborhood of the other.
        The bigger epsilon, the larger the clusters will be on average; we want to tune epsilon so that points
        within a branch have distance < eps, while points from different branches have distance > eps.
    min_samples : the minimum number of samples that have to be in the neighborhood of a point
        for it to be considered a "core sample"

    Returns two dictionaries with the cluster centroids as keys: one with the points belonging to each cluster as values, 
    and the other with the average filter value of the points in the cluster as value
    '''
    dbscan = DBSCAN(eps=eps, min_samples=min_samples).fit(covering_set)
    labels = dbscan.labels_  # Note: DBSCAN also labels some points as "noise" (-1)

    output_pts = {}
    output_vals = {}
    # For each cluster, calculate the cluster centroid as the mean of the points in that cluster (this is not done by DBSCAN)
    # and assign all points in the cluster to that centroid in the output dictionary.
    # Note: points labeled as noise are not included in any cluster.
    for i in np.unique(labels):
        if i < 0:
            # If we wanna do something with the noise points, do it here
            continue
        cluster_points = covering_set[labels == i]
        centroid = np.average(cluster_points, axis=0)
        output_pts[tuple(centroid)] = cluster_points
        output_vals[tuple(centroid)] = np.mean(covering_set_vals[labels == i])

    return output_pts, output_vals

# TEST
X = np.array([[0,0,0], [1,1.5,1], [0,0.5,0], [1,1,1], [1.5,1,1], [0.5,0,0]])
X_vals = np.array([0, 1.5, 0.5, 1, 1, 0])
#clusters, vals = cluster_kmeans(X, X_vals, 2)
clusters, vals = cluster_linkage(X, X_vals, 1)
#clusters, vals = cluster_dbscan(X, X_vals, 0.5, 3)
print(clusters)
print(vals)

{(1.1666666666666667, 1.1666666666666667, 1.0): array([[1. , 1.5, 1. ],
       [1. , 1. , 1. ],
       [1.5, 1. , 1. ]]), (0.16666666666666666, 0.16666666666666666, 0.0): array([[0. , 0. , 0. ],
       [0. , 0.5, 0. ],
       [0.5, 0. , 0. ]])}
{(1.1666666666666667, 1.1666666666666667, 1.0): 1.1666666666666667, (0.16666666666666666, 0.16666666666666666, 0.0): 0.16666666666666666}


### Generate graph 

In [28]:
### GENERATE GRAPH ###
# Compute edges between clusters based on common points, and use this to produce a graph, visualized using the PyVis library.
# If the filter value of a point x is in the overlap of intervals I and J, then there is a cluster in the pre-image of I
# and a cluster in the pre-image of J that both contain x; these clusters will then be connected.
# Note: since every point in a covering set is in *one* unique cluster within that covering set, there are no edges
# between clusters within the same covering set (this is the whole point; to detect different connected components)

import pyvis as pv
import matplotlib.cm as cm 
from matplotlib.colors import rgb2hex

def compute_graph(cluster_dict, cluster_dict_vals, colorby='value', label_args={}):
    '''
    Converts the given list of clusters into a PyVis network.

    Parameters:
    - cluster_dict : a nested dictionary of the following form:
        {cover_interval_1: {cluster_1: [point1, point2, ...], cluster_2: [point5, point7, ...], ..., cluster_k1: [point4, point12, ...]},
        cover_interval_2: {cluster_1: [point1, point56, ...], cluster_2: [point32, point4, ...], ..., cluster_k2: [point41, point95, ...]},
        ...}
    - cluster_dict_vals : a nested dictionary with the corresponding average filter values for each cluster:
        {cover_interval_1: {cluster_1: avg_value_of_points_in_cluster_1, cluster_2: avg_value_of_points_in_cluster_2, ..., cluster_k1: avg_value_of_points_in_cluster_k1},
        cover_interval_2: {cluster_1: avg_value_of_points_in_cluster_1, cluster_2: avg_value_of_points_in_cluster_2, ..., cluster_k2: avg_value_of_points_in_cluster_k2},
        ...}
    - colorby : 'coverset', 'value' or 'label'
        Whether to color the nodes by the index of their cover interval (coverset), by their average filter value (value),
        or by their label given by height and branch dir (label)
    - label_args : arguments for the label function. Only used if colorby='label'.
        A dictionary with keys 'trunk_threshold', 'support_threshold' and 'max_growth_angle'.

    Returns:
    - graph: a PyVis Network object. Nodes can be accessed via graph.nodes and edges via graph.edges.
    '''
    graph = pv.network.Network(notebook=True, cdn_resources='in_line')
    graph.toggle_physics(False)  # Disable the physics-based layout since we want to put nodes at custom positions (namely cluster centroids)
    colormap = cm.get_cmap('viridis')
    nr_of_coversets = len(cluster_dict_vals.values())

    fmin = np.min([value for coverset in cluster_dict_vals.values() for value in list(coverset.values())])
    fmax = np.max([value for coverset in cluster_dict_vals.values() for value in list(coverset.values())])

    if(colorby == 'coverset'):
        color_fn = lambda filter_val, i : rgb2hex(colormap(i/nr_of_coversets))
    elif(colorby == 'value' or colorby == 'branch_direction'):
        color_fn = lambda filter_val, i : rgb2hex(colormap((filter_val-fmin)/(fmax-fmin)))
    else:
        color_fn = lambda filter_val, i : label_fn(filter_val, i, nr_of_coversets, label_args['trunk_threshold'], label_args['support_treshold'], label_args['max_growth_angle'])

    # Add nodes; these are the centroid clusters
    for i, coverset in enumerate(cluster_dict_vals.values()):
        nodes = list(coverset.keys())         # = Cluster centroids
        filter_vals = list(coverset.values()) # = Avg filter values of points in clusters
        graph.add_nodes(
            ['{}-{}'.format(i, j) for j in range(len(nodes))],             # ID = coverset_nr-cluster_nr
            color=[color_fn(filter_val, i) for filter_val in filter_vals], # Color by coverset or value
            x = [node[0]*1000 for node in nodes],                          # Set x,y position manually as centroid position (*1000 to show them properly separated)
            y = [node[1]*1000 for node in nodes]
        ) 

    # Add edges (can only safely be done after all nodes have been added)
    # For each cluster, see if any of the cluster's points can also be found in any other clusters; if so, add an edge.
    # Since every point within a coverset has one unique cluster, we don't have to check the cluster's own coverset.
    # Also, since the graph is undirected, we don't need to check previous coversets.
    # But we are still left with a 4-layer for loop; maybe see if we can implement this more efficiently.
    for iA, coversetA in tqdm(enumerate(list(cluster_dict.values())[:-1])): # Don't need to check last coverset since graph is undirected
        for jA, clusterA in enumerate(coversetA.values()):
            #print("Cluster A ({}-{}): ".format(iA, jA), clusterA) # DEBUG
            # See which clusters in other coversets share an index with this cluster
            for iB, coversetB in enumerate(list(cluster_dict.values())[iA+1:]): # Previous coversets have already been checked
                iB += iA+1  # Correct index
                for jB, clusterB in enumerate(coversetB.values()):
                    #print("\t Cluster B ({}-{}): ".format(iB, jB), clusterB) # DEBUG
                    #print("\t Common points: ", arrays_intersect(clusterA, clusterB)) # DEBUG
                    if arrays_intersect(clusterA, clusterB):
                        graph.add_edge('{}-{}'.format(iA, jA), '{}-{}'.format(iB, jB))

    return graph

def arrays_intersect(A, B):
    '''Returns true iff arrays A and B have at least one element in common.'''
    # any(point in clusterB for point in clusterA) # This returns incorrect results because numpy is funny :tm:
    for x in A:
        if np.any(np.all(x == B, axis=1)): # If all coordinates of x match with all coordinates of any point in B, return True
            return True
    return False

# Function for labeling branches. Basic idea:
# - Points in the bottom (by height) trunk_threshold% of coversets are labeled as trunk.
# - Points in the bottom support_threshold% that are not in the bottom trunk_threshold% are labeled as support.
# - For the other points, if the y-component of their branch growth direction is small (i.e. the branch grows near-horizontally around them),
#   they are labeled as side branch; otherwise, they are labeled as leader branches.
# Colors are consistent with those used in the paper.
trunk_color = "#605f29"
support_color = "#e38888"
leader_color = "#4a5ba8"
side_color = "#14a64d"

def label_fn(growth_dir, coverset_nr, nr_of_coversets, trunk_threshold, support_treshold, max_growth_angle):
    if coverset_nr > nr_of_coversets * (1 - trunk_threshold):
        return trunk_color
    elif coverset_nr > nr_of_coversets * (1 - support_treshold):
        return support_color
    elif growth_dir < max_growth_angle:
        return side_color
    else:
        return leader_color

# TEST (filter value is y-coordinate)
X = {(0, 0.7): np.array([[0.1, 0.1, 0.4], [0, 0.2, 0.5], [0.7, 0.5, -0.2], [0.8, 0.6, -0.1]]),
     (0.3, 1): np.array([[0.7, 0.5, -0.2], [0.8, 0.6, -0.1], [0.2, 0.9, 0.8], [0.1, 0.8, 0.9]]),
     (0.6, 1.3): np.array([[0.8, 0.6, -0.1], [0.2, 0.9, 0.8], [0.1, 0.8, 0.9], [-0.6, 1.2, -0.5], [-0.7, 1.1, -0.4]])}
X_vals = {(0, 0.7): np.array([0.1, 0.2, 0.5, 0.6]),
     (0.3, 1): np.array([0.5, 0.6, 0.9, 0.8]),
     (0.6, 1.3): np.array([0.6, 0.9, 0.8, 1.2, 1.1])}
cluster_pts = {}
cluster_vals = {}
for cover_interval in X.keys():
    points = X[cover_interval]
    vals = X_vals[cover_interval]
    cluster_pts[cover_interval], cluster_vals[cover_interval] = cluster_kmeans(points, vals, 2)
print(cluster_pts)
print(cluster_vals)

#compute_graph(cluster_dict)

{(0, 0.7): {(0.75, 0.55, -0.15): array([[ 0.7,  0.5, -0.2],
       [ 0.8,  0.6, -0.1]]), (0.04999999999999999, 0.15000000000000002, 0.44999999999999996): array([[0.1, 0.1, 0.4],
       [0. , 0.2, 0.5]])}, (0.3, 1): {(0.15000000000000002, 0.8500000000000001, 0.85): array([[0.2, 0.9, 0.8],
       [0.1, 0.8, 0.9]]), (0.75, 0.55, -0.15000000000000002): array([[ 0.7,  0.5, -0.2],
       [ 0.8,  0.6, -0.1]])}, (0.6, 1.3): {(0.36666666666666664, 0.7666666666666667, 0.5333333333333334): array([[ 0.8,  0.6, -0.1],
       [ 0.2,  0.9,  0.8],
       [ 0.1,  0.8,  0.9]]), (-0.65, 1.15, -0.45000000000000007): array([[-0.6,  1.2, -0.5],
       [-0.7,  1.1, -0.4]])}}
{(0, 0.7): {(0.75, 0.55, -0.15): 0.55, (0.04999999999999999, 0.15000000000000002, 0.44999999999999996): 0.15000000000000002}, (0.3, 1): {(0.15000000000000002, 0.8500000000000001, 0.85): 0.8500000000000001, (0.75, 0.55, -0.15000000000000002): 0.55}, (0.6, 1.3): {(0.36666666666666664, 0.7666666666666667, 0.5333333333333334): 0.766666666666

In [29]:
import open3d as o3d

def overlay_graph_on_cloud(pcd, cluster_dict):
    # 1) Build a flat list of (centroid, points) and assign each an index
    nodes = []
    for coverset in cluster_dict.values():
        for centroid, pts in coverset.items():
            nodes.append((np.array(centroid), pts))

    centroids = np.vstack([c for c, _ in nodes])

    # 2) Compute edges: connect if clusters from different coversets share ≥1 point
    edges = []
    for iA, (cA, ptsA) in enumerate(nodes):
        for iB, (cB, ptsB) in enumerate(nodes[iA+1:], start=iA+1):
            # fast numpy check
            # each row of ptsA vs each row of ptsB
            if np.any([np.all(x==ptsB,axis=1).any() for x in ptsA]):
                edges.append([iA, iB])

    # 3) Create Open3D geometries
    # 3a) original cloud (with its own colors, etc.)
    cloud = pcd  # already loaded & voxel-downsampled before calling mapper()

    # 3b) centroids as small red points
    cent_pc = o3d.geometry.PointCloud()
    cent_pc.points = o3d.utility.Vector3dVector(centroids)
    cent_pc.paint_uniform_color([1, 0, 0])  # red nodes

    # 3c) edges as green lines
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(centroids),
        lines=o3d.utility.Vector2iVector(edges)
    )
    line_set.colors = o3d.utility.Vector3dVector([[0,1,0] for _ in edges])

    # 4) Visualize all together
    o3d.visualization.draw_geometries([cloud, cent_pc, line_set])

### Full algorithm

In [30]:
### FULL MAPPER ALGORITHM ##
def mapper(X, X_branch_dirs, org_pcd, filter_alg='height', cluster_alg = 'kmeans', cluster_args={}, cover_l=0.4, cover_p=0.4, colorby='value', label_args={}, bag_nr = 1):
    '''Runs the mapper algorithm on point cloud dataset X. Make sure to have run the cells above.
    
    Parameters
    ------------
    X : The point cloud dataset. Points are assumed to be arrays of the form [x,y,z].
    X_branch_dirs : Branch direction vectors corresponding to the points, also of the form [x,y,z].
    org_pcd : the original point cloud
    filter_alg : 'height', 'branch_direction', 'density', 'eccentricity' or 'knn_distance'
        The filter to use. 
    cluster_alg : 'kmeans', 'linkage' or 'dbscan'
        The clustering algorithm to use.
    cluster_args : dict with arguments to use for the chosen clustering function.
    cover_l : float, length of intervals used to cluster the parameter space R.
    cover_p : float in [0,1], percentage overlap between intervals.
    colorby : 'coverset', 'value', 'branch_direction' or 'label'
        Whether to color nodes in the resulting graph by coverset, by filter value or by branch direction. Effectively, 'coverset' means all nodes/clusters whose points have a similar filter value (i.e. which are in the same covering set) will get the *same* color,
        while in 'value', the color is instead based on the average filter value of the points within a cluster; clusters of the same covering set will have similar, but non-identical colors, especially if the covering intervals are large.
        If colorby='branch_direction', the nodes are colored based on the branch direction filter values, regardless of which filter is used for filtering.
        Finally, if colorby='label', the nodes are colored based on a labeling function, which is in turn based on the height and branch direction.
    label_args : arguments for the labeling function. Only used if colorby='label'.
    bag_nr : index of the tree (included in file name in order to not overwrite files for different bags)
    
    Outputs
    ------------
    Creates a file mapper_output.html containing the output graph. 
    If it does not open automatically, open it in your browser manually.
    '''
    print("### INITIALIZING MAPPER ###")
    file_name = 'MAPPER_BAG={}'.format(bag_nr)  # Parameter values are stored in the filename

    # 1: Calculate filter values
    filter_vals = []
    match filter_alg:
        case 'height':
            for point in X:
                filter_vals.append(filter_height(point))
            file_name += '_FILTER=height'
        case 'branch_direction':
            for branch_dir in X_branch_dirs:
                filter_vals.append(branch_direction_filter(branch_dir))
            file_name += '_FILTER=branchdirection'
        case 'density':
            kernel = 'gaussian'
            bandwidth = 1
            filter_vals = estimate_local_point_density(X, kernel, bandwidth)
            file_name += '_FILTER=density_kernel={}_bandwidth={}'.format(kernel, bandwidth)
        case 'eccentricity':
            p = 2
            filter_vals = eccentrity_filtering(X, p)
            file_name += '_FILTER=eccentricity_p={}'.format(p)
        case 'knn_distance':
            k = 10
            filter_vals = knn_distance(X, k)
            file_name += '_FILTER=knndistance_k={}'.format(k)
        case _:
            raise Exception("Filter function not recognized!")

    fmin = min(filter_vals)
    fmax = max(filter_vals)
    #print("Filter values: ", filter_vals) # DEBUG

    # To color/labelpoints by branch direction while using a different filter function, 
    # we produce X_cover based on said filter function, while using the branch direction values for X_cover_vals.
    branch_dir_vals = []
    if(colorby == 'branch_direction' or colorby == 'label'):
        for branch_dir in X_branch_dirs:
            branch_dir_vals.append(branch_direction_filter(branch_dir))
    
    # 2: Cover parameter space with open intervals
    cover_dict = cover_intervals(fmin, fmax, cover_l, cover_p)
    file_name += '_PARTITION=intervals_l={}_p={}'.format(cover_l, cover_p)
    #print("Cover of parameter space: ", cover_dict) # DEBUG

    # 3: Assign points in X to covering sets based on these intervals
    print("GENERATING COVER FOR X")
    X_cover, X_cover_vals = apply_covering(X, filter_vals, cover_dict, branch_dir_vals)
    print("Cover of X: ", X_cover) # DEBUG
    print("Corresponding filter vals: ", X_cover_vals) # DEBUG

    # 4: Apply clustering on each covering set
    print("CLUSTERING COVERING SETS")

    # Select clustering algorithm based on user input
    match cluster_alg: 
        case 'kmeans':
            k = cluster_args['k']
            cluster_fn = lambda points, vals : cluster_kmeans(points, vals, k)
            file_name += '_CLUSTER=kmeans_k={}'.format(k)
        case 'linkage':
            distance_threshold = cluster_args['distance_threshold']
            cluster_fn = lambda points, vals : cluster_linkage(points, vals, distance_threshold)
            file_name += '_CLUSTER=linkage_dthreshold={}'.format(distance_threshold)
        case 'dbscan':
            eps = cluster_args['eps']
            min_samples = cluster_args['min_samples']
            cluster_fn = lambda points, vals : cluster_dbscan(points, vals, eps, min_samples)
            file_name += '_CLUSTER=dbscan_eps={}_minsamples={}'.format(eps, min_samples)
        case _:
            raise Exception("Clustering algorithm not recognized!")
    
    X_clustered = {}
    X_clustered_vals = {}
    for cover_interval in tqdm(cover_dict.keys()):
        points = X_cover[cover_interval]
        vals = X_cover_vals[cover_interval]
        X_clustered[cover_interval], X_clustered_vals[cover_interval] = cluster_fn(np.array(points), np.array(vals))
    #print("X with covering sets clustered: ", X_clustered) # DEBUG
    #print("Corresponding values: ", X_clustered_vals) # DEBUG

    # 5: Use clusters to compute mapper graph
    print("COMPUTING GRAPH")
    file_name += '_COLORBY={}'.format(colorby)
    file_name += '.html'
    graph = compute_graph(X_clustered, X_clustered_vals, colorby=colorby, label_args=label_args)
    graph.prep_notebook()

    # Wouter: @Kishan or whoever changed this, this does not work, at least not in my VSCode. I'm reverting back to the original graph.show.
    # Manually write out with UTF-8
    #out_path = f'mapper_outputs/{file_name}'
    #with open(out_path, 'w', encoding='utf-8') as f:
    #    f.write(graph.html)
    #print(f"Wrote mapper graph to {out_path}")

    graph.show('mapper_outputs/' + file_name, notebook=True)

    print("Succesfully generated graph ", file_name)
    print("If it does not open automatically, open the html in your browser.")
    print("###########################")

    # @Kishan add function here to show the graph overlayed over the point cloud.
    # Nodes correspond to clusters and should be plotted in 3D at the cluster center's coordinate.
    overlay_graph_on_cloud(org_pcd, X_clustered)

# TEST
#X = [[0.1, 0.1, 0.4], [0, 0.2, 0.5], [0.7, 0.5, -0.2], [0.8, 0.6, -0.1], 
#     [0.2, 0.9, 0.8], [0.1, 0.8, 0.9], [-0.6, 1.2, -0.5], [-0.7, 1.1, -0.4]]
#mapper(X)

# Experiments

### Parameters

In [ ]:
import open3d as o3d

### MODIFY EXPERIMENT PARAMETERS HERE ###
BAG_NR = 27                        # Which tree (point cloud) to look at, between 0 and 83 (both inclusive)

# PREPROCESSING
REDO_PREPROCESSING = True         # If True, reruns preprocessing every time; if False, skips preprocessing if there is already a point cloud in memory
                                   # NOTE: It would be better to do this with a save/load mechanism, but idk how exactly to do this :-(

USE_STAT_OUTLIER_REMOVAL = True    # Whether to use statistical outlier removal in preprocessing (removes points if the distance to their neighbours is further than the average neighbour-distance in the point cloud)
SOR_NB_NEIGHBOURS = 50             # |-> Nr of neighbours that are taken into account to calculate average distance
SOR_STD_RATIO = 3                # |-> How many STDs from the average a point's neighbour-distance needs to be in order to be removed

USE_RADIUS_OUTLIER_REMOVAL = False # Whether to use radius outlier removal; removes points that have few neighbours in a sphere around them.
ROR_NB_POINTS = 50                 # |-> If a point has fewer than this nr of neighbours, it will be removed
ROR_RADIUS = 0.02                  # |-> Radius of the sphere that we check in

USE_VOXEL_DOWNSAMPLING = True      # Whether to use voxel downsampling; divides the space into cubical buckets (voxels), and aggregates the points in a voxel by averaging them into a single point
VOXEL_RADIUS = 0.002                # |-> Radius/side length of the voxels

R_SUPER = 0.06                     # For superpoint downsampling: the radius of the superpoint (points within this radius are removed from the dataset).
R_BRANCH = 0.1                     # For calculating branch growth direction; normals of points within this radius around the superpoint will be considered for calculating the branch dir. 
                                   # This should roughly be the width of a branch.

NORMAL_K = 20                      # For normal vector generation: nr of neighbours to consider when generating the normal
# Note: superpoint downsampling and normal vector generation are always used, since these are necessary for the algorithm to function.

# MAPPER
FILTER_ALG = 'height'              # Which filter to use
CLUSTER_ALG = 'dbscan'             # Which clustering algorithm to use
COVER_L = 0.3                      # Length of covering intervals
COVER_P = 0.4                      # Percentage overlap of covering intervals
COLORBY = 'label'                  # Whether to color graph nodes by coverset, filter value, branch direction or label

DBSCAN_EPS = 0.08        # For DBSCAN: max distance between two samples for one to be considered in the neighborhood of the other.
DBSCAN_MIN_SAMPLES = 2   # the minimum number of samples that have to be in the neighborhood of a point for it to be considered a "core sample"

TRUNK_THRESHOLD = 0.3     # For the labeling function: Points in the bottom trunk_threshold% coversets are labeled as trunk.
SUPPORT_THRESHOLD = 0.4   # Points in coverset >= trunk_threshold% but < support_threshold% are labeled as support.
MAX_GROWTH_ANGLE = 0.4    # Otherwise, if abs y component of the growth direction is < this number, label as side branch, otherwise leader branch.
#########################################

# Load point cloud
FILE = "data/bag_{}/cloud_final.ply".format(BAG_NR)
pcd = o3d.io.read_point_cloud(FILE)
np_pcd = len(pcd.points)
print(pcd) 

CLUSTER_ARGS = {'eps': DBSCAN_EPS, 'min_samples': DBSCAN_MIN_SAMPLES}
LABEL_ARGS = {'trunk_threshold': TRUNK_THRESHOLD, 'support_treshold': SUPPORT_THRESHOLD, 'max_growth_angle': MAX_GROWTH_ANGLE}

PointCloud with 353443 points.


### Preprocessing
To use normal vectors as input to mapper, the order should be

Load pointcloud > Remove outliers > Generate normal vector for each point (based on entire pointcloud) > Sample superpoints; at the same time, aggregate the normal vectors around this superpoint into a single normal vector, which will be attached to this superpoint > Run mapper; label resulting nodes based on their filter value.

Alternative, if we use the normal vectors only for labeling, not filtering, then it would instead be

Load pointcloud > Remove outliers > Generate normal vector for each point (based on entire pointcloud) > Sample superpoints > Run mapper > Aggregate normal vectors within a cluster; label based on the direction of the aggregated vector.

In [104]:
import copy
import numpy as np
from sklearn.decomposition import PCA

# ---- 1. Statistical Outlier Removal ----
def statistical_outlier_removal(pcd, nb_neighbors, std_ratio):
    pcd_stat = copy.deepcopy(pcd)
    pcd_stat, _ = pcd_stat.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
    num_points = len(pcd_stat.points)
    return pcd_stat, num_points

# VISUALIZATION
#pcd_stat, num_stat_points = statistical_outlier_removal(pcd_original, nb_neighbors=50, std_ratio=0.3)
#pcd_stat.paint_uniform_color([0, 0.8, 0.2])  # green
#pcd_stat.translate((0.02, 0, 0))

# print("After Statistical Outlier Removal:", num_stat_points)
#o3d.visualization.draw_geometries(
#    [pcd_original, pcd_stat],
#    window_name="Voxel Downsampled vs Statistical Outlier Removal",
#    width=960, height=720
#)

#std_ratios = np.linspace(0.05, 0.5, 10)
#num_voxel_points_radius = np.full(10, np.nan)
#for i, std_ratio in enumerate(std_ratios):
#    _, num_radius_points = statistical_outlier_removal(pcd_original, nb_neighbors=50, std_ratio=std_ratio)
#    num_voxel_points_radius[i] = num_radius_points 

#fig, ax = plt.subplots(figsize=(10, 10))
#ax.plot(std_ratios, num_voxel_points_radius, label="Num. Points")
#ax.set_yscale('log')
#ax.set_ylim([0, np_original_pcd])
#ax.set_xlabel("Voxel Radius")
#ax.set_ylabel("Number of points")
#plt.show()

# ---- 2. Radius Outlier Removal ----
def radius_outlier_removal(pcd, nb_points, radius):
    pcd_radius = copy.deepcopy(pcd)
    pcd_radius, _ = pcd_radius.remove_radius_outlier(nb_points=nb_points, radius=radius)
    num_points = len(pcd_radius.points)
    return pcd_radius, num_points

# VISUALIZATION
#pcd_radius, num_radius_points = radius_outlier_removal(pcd_original, nb_points=50, radius=0.02)
# print("After Radius Outlier Removal:", num_radius_points)
#pcd_radius.paint_uniform_color([0, 0.8, 0.2])  # green
#pcd_radius.translate((0.02, 0, 0))

#o3d.visualization.draw_geometries(
#    [pcd_original, pcd_radius],
#    window_name="Voxel Downsampled vs Radius Outlier Removal",
#    width=960, height=720
#)

#radius_sizes = np.linspace(0.005, 0.05, 10)
#num_voxel_points_radius = np.full(10, np.nan)
#for i, voxel_radius in enumerate(radius_sizes):
#    _, num_radius_points = radius_outlier_removal(pcd_original, nb_points=50, radius=voxel_radius)
#    num_voxel_points_radius[i] = num_radius_points 

#fig, ax = plt.subplots(figsize=(10, 10))
#ax.plot(radius_sizes, num_voxel_points_radius, label="Num. Points")
#ax.set_yscale('log')
#ax.set_ylim([0, np_original_pcd])
#ax.set_xlabel("Voxel Radius")
#ax.set_ylabel("Number of points")
#plt.show()

# ---- 3. Voxel ----
def voxel_downsampling(pcd, voxel_radius):
    pcd_voxel = copy.deepcopy(pcd)
    pcd_voxel = pcd_voxel.voxel_down_sample(voxel_radius)
    num_points = len(pcd_voxel.points)
    return pcd_voxel, num_points

# VISUALIZATION
#pcd_voxel, num_voxel_points = voxel_downsampling(pcd_original, 0.02)
# print(f"After Voxel Downsampling for radius {0.02}:", num_voxel_points)
#pcd_voxel.paint_uniform_color([0, 0.8, 0.2])  # green
#pcd_voxel.translate((0.02, 0, 0))

#o3d.visualization.draw_geometries(
#    [pcd_original, pcd_voxel],
#    window_name="Orginal vs Voxel Downsampled Point clouds",
#    width=960, height=720
#)

#voxel_sizes = np.linspace(0.01, 0.2, 20)
#num_voxel_points_radius = np.full(20, np.nan)
#for i, voxel_radius in enumerate(voxel_sizes):
#    _, num_voxel_points = voxel_downsampling(pcd_original, voxel_radius)
#    num_voxel_points_radius[i] = num_voxel_points 

#fig, ax = plt.subplots(figsize=(10, 10))
#ax.plot(voxel_sizes, num_voxel_points_radius, label="Num. Points")
#ax.set_yscale('log')
#ax.set_ylim([0, np_original_pcd])
#ax.set_xlabel("Voxel Radius")
#ax.set_ylabel("Number of points")
#plt.show()

# ---- 4. Normal Estimation ----
def generate_normals(pcd, k=30):
    '''Generates a normal vector for each point based on its k nearest neighbours'''

    # Estimate normals
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=k)) # pcd will now have the points in pcd.points and the corresponding normals in pcd.normals

    # Normalize normals (optional)
    pcd.normalize_normals()

    return pcd, len(pcd.points)

# VISUALIZATION
# Parameters for arrow size
#arrow_length = 0.005

# Create lines representing normal vectors
#points = np.asarray(pcd_normal.points)
#normals = np.asarray(pcd_normal.normals)

# Start and end points of arrows
#line_points = []
#line_indices = []

#for i, (p, n) in enumerate(zip(points, normals)):
#    line_points.append(p)
#    line_points.append(p + arrow_length * n)
#    line_indices.append([2 * i, 2 * i + 1])

# Create LineSet for arrows
#arrow_lines = o3d.geometry.LineSet()
#arrow_lines.points = o3d.utility.Vector3dVector(line_points)
#arrow_lines.lines = o3d.utility.Vector2iVector(line_indices)

# Optional: set color of arrows (red)
#arrow_lines.colors = o3d.utility.Vector3dVector([[1, 0, 0] for _ in line_indices])

# Visualize
#o3d.visualization.draw_geometries(
#    [pcd_normal, arrow_lines],
#    window_name="Point Normals as Arrows",
#    width=960, height=720
#)

# --- ESTIMATE BRANCH GROWTH DIRECTION FROM NORMALS ---
# The normal vectors contain information about the branch's growth direction, but the vectors themselves are noisy. 
# Our situation is as follows: a branch is roughly a cilinder, so the normal vectors of the points along the surface will roughly point radially outward from the branch's center. 
# Simply averaging them won't work, since then the resulting vector would be 0. Instead, we want to find a vector that is "roughly perpendicular" to all vectors in a region (i.e. a ball whose radius is roughly the radius of the branch).
# After doing some research, the best way to do this seems to be **fitting a plane to the normal vectors** and then taking the normal of this plane; this will (roughly, locally) coincide with the branch's growth direction. 
# Fitting a plane like this can be done using PCA or similar methods. Since the vectors can be quite noisy, it might be a good idea to add some outlier removal/robustness to the PCA implementation.

def find_branch_direction(normals):
    '''Returns a vector that is roughly perpendicular to the provided normal vectors (assumed to be normalized).
    Note: it is assumed that "normals" is only a small region of the point cloud. Selecting this region should be done outside this function.
    This function can be used either to find the growth direction around a superpoint (in which case the region should be a neighbourhood of the superpoint)
    or to find the growth direction of a node in the mapper output (in which case the region is all points in the corresponding cluster).'''
    branch_direction = np.array([0,0,0])
    # We can only fit a plane if we have at least 3 samples. Otherwise, the growth direction will be 0 (reasonable, as we simply don't have enough information to infer a direction)
    if(len(normals) > 2):
        pca = PCA(n_components=3)
        pca.fit(normals)
        # Add outlier removal here (first see how well the basic version works though)

        # The branch direction is the least significant component, i.e. the direction in which the normals have the least variance, i.e. the normal to the plane spanned by the two most significant components.
        branch_direction = pca.components_[-1] 
    return branch_direction

# ---- 5. Superpoint selection & Branch direction determination ----
def superpoint_selection(pcd_with_normals, r_super=0.05, r_branch=0.1):
    '''Generates superpoints from a pointcloud with normals.
    - A superpoint "represents" its neighbourhood; all points within r_super of the superpoint are discarded. Superpoints are selected in this way until only superpoints remain, and all other points have been removed.
    - The vector corresponding to a superpoint is the aggregate of the normals of the points represented by the superpoint, as given by the filter_branch_direction function. It represents the local growth direction of the branch.

    Returns:
    - Super points (np array)
    - Corresponding branch direction vectors (np array)
    - Nr of superpoints (int)
    '''
    points = np.asarray(pcd_with_normals.points)
    normals = np.asarray(pcd_with_normals.normals)

    bool_pts = np.zeros(points.shape[0], dtype=bool) # Keeps track of which points have been represented
    super_points = []
    super_point_normals = []
    while not np.all(bool_pts):
        remaining_indices = np.where(~bool_pts)[0] # Remaining indices
        print(np.sum(~bool_pts))

        pts_remain = points[~bool_pts]      # Uncovered points
        rand_super_pt = pts_remain[np.random.choice(pts_remain.shape[0])] # Preliminary superpoint point chosen randomly from uncovered points

        bool_pts_super_pt = np.sum(np.abs(pts_remain - rand_super_pt), axis=1) < r_super # Indices of points represented by this superpoint
        bool_pts_branch_dir = np.sum(np.abs(points - rand_super_pt), axis=1) < r_branch # Points (covered or uncovered) used for determining the branch direction around this superpoint

        super_pt = np.mean(pts_remain[bool_pts_super_pt], axis=0) # Actual superpoint will be the average of points covered by the preliminary choice
        super_points.append(super_pt)

        super_pt_normal = find_branch_direction(normals[bool_pts_branch_dir]) # Corresponding branch dir vector will be approximately perpendicular to all vectors in the radius
        super_point_normals.append(super_pt_normal)

        bool_pts[remaining_indices[bool_pts_super_pt]] = True # Change set of Covered Points accordingly

    return np.array(super_points), np.array(super_point_normals), len(super_points)

In [105]:
# Actually do preprocessing
print(f"Original nr of points: {np_pcd}")

def preprocess_pointcloud(pcd):
    if(USE_STAT_OUTLIER_REMOVAL):
        pcd, np_pcd = statistical_outlier_removal(pcd, SOR_NB_NEIGHBOURS, SOR_STD_RATIO)
        print(f"Remaining after statistical outlier removal: {np_pcd}")

    if(USE_RADIUS_OUTLIER_REMOVAL):
        pcd, np_pcd = radius_outlier_removal(pcd, ROR_NB_POINTS, ROR_RADIUS)
        print(f"Remaining after radius outlier removal: {np_pcd}")

    if(USE_VOXEL_DOWNSAMPLING):
        pcd, np_pcd = voxel_downsampling(pcd, VOXEL_RADIUS)
        print(f"Remaining after voxel downsampling: {np_pcd}")

    # Generate normal vectors for remaining points
    pcd_normal, _ = generate_normals(pcd, NORMAL_K)

    # Do superpoint selection
    pcd_super_points, pcd_super_normals, np_pcd_super = superpoint_selection(pcd_normal, R_SUPER, R_BRANCH)

    return pcd, pcd_super_points, pcd_super_normals, np_pcd_super # pcd is original point cloud after outlier removal but before normal generation


if(REDO_PREPROCESSING):
    pcd, pcd_super_points, pcd_super_normals, np_pcd_super = preprocess_pointcloud(pcd)
else: # Don't redo preprocessing if we already have a point cloud in memory
    try:
        print(f"Already generated superpoints: {pcd_super_points}")
    except:
        pcd, pcd_super_points, pcd_super_normals, np_pcd_super = preprocess_pointcloud(pcd)

print("Nr of superpoints: ", np_pcd_super)


Original nr of points: 353443
Remaining after statistical outlier removal: 350386
Remaining after voxel downsampling: 345000
345000
344817
343654
343449
342766
342113
341644
341427
341078
340581
339510
339073
338181
337639
337401
336524
336264
336022
335463
335046
334675
334415
334290
334028
333277
333013
332943
332454
332391
332295
331922
331559
330327
330216
329942
329782
329049
328552
328199
327607
327377
327245
327128
326115
325207
325054
325005
324670
324211
324099
323830
323511
322513
322368
321307
320779
320460
320441
319634
319476
319278
318442
318191
317687
317440
317279
316877
316789
316467
316224
315742
314902
314681
314206
313957
313887
313448
312872
312181
312083
311898
311594
311524
310829
310760
310542
310109
309668
309464
309156
308490
307900
307460
307171
306687
306631
306339
306168
306096
305270
305015
304701
303821
303554
303303
303032
302621
302170
301894
301487
301405
301326
300570
299610
299545
299179
298887
298671
298093
297554
297058
296870
296140
296006
295539


### Run mapper

In [106]:
# Run mapper algorithm
mapper(pcd_super_points, pcd_super_normals, pcd, filter_alg=FILTER_ALG, cluster_alg=CLUSTER_ALG, cluster_args=CLUSTER_ARGS, 
       cover_l=COVER_L, cover_p=COVER_P, colorby=COLORBY, label_args=LABEL_ARGS, bag_nr=BAG_NR)

### INITIALIZING MAPPER ###
GENERATING COVER FOR X


3191it [00:00, 175323.54it/s]

Cover of X:  

{(-0.8196766817857143, -0.5196766817857144): [array([ 1.00825833, -0.59455168,  1.24793751]), array([-0.0615741 , -0.58300476,  1.27570525]), array([-0.0540107 , -0.51996072,  1.29122659]), array([ 0.59701619, -0.79108306,  1.23128878]), array([ 0.44421725, -0.6927394 ,  1.2189525 ]), array([ 0.41392044, -0.62455399,  1.24120436]), array([ 0.87920885, -0.78181505,  1.22260367]), array([-0.71654113, -0.55096605,  1.28879709]), array([ 0.9772236 , -0.78568941,  1.20081554]), array([-0.07237133, -0.71031957,  1.21824308]), array([-0.0609785 , -0.65394132,  1.24538596]), array([ 0.40499361, -0.56274072,  1.27498105]), array([ 0.58466078, -0.77917943,  1.3237034 ]), array([ 0.43572939, -0.58393887,  1.26409091]), array([ 0.98284862, -0.72111383,  1.21661265]), array([-0.07255177, -0.79473162,  1.19328764]), array([ 0.41328731, -0.7972157 ,  1.20575471]), array([ 0.99965939, -0.65343851,  1.23476251]), array([-0.35076202, -0.67664217,  1.25058882]), array([ 0.78697356, -0.68005757,  1.132684

100%|██████████| 12/12 [00:00<00:00, 136.17it/s]
C:\Users\woute\AppData\Local\Temp\ipykernel_22236\977487694.py:36: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap('viridis')


COMPUTING GRAPH


11it [00:10,  1.09it/s]


mapper_outputs/MAPPER_BAG=27_FILTER=height_PARTITION=intervals_l=0.3_p=0.4_CLUSTER=dbscan_eps=0.08_minsamples=2_COLORBY=label.html
Succesfully generated graph  MAPPER_BAG=27_FILTER=height_PARTITION=intervals_l=0.3_p=0.4_CLUSTER=dbscan_eps=0.08_minsamples=2_COLORBY=label.html
If it does not open automatically, open the html in your browser.
###########################
